# Emergency Presentation analysis for paper

In [22]:
import pyodbc
import pandas as pd  # sure takes a long time

## SQL setup

In [28]:
conn = pyodbc.connect("DRIVER={SQL Server};SERVER=VHACDWRB03;DATABASE=ORD_Singh_202001030D")

# other stuff you can do with a connection but we won't
# cursor = conn.cursor()
# cursor.description  # gives you column names
# cursor.execute(sql)
#for i, row in enumerate(cursor):
#    if i < 20:
#        print(row)
# conn.close()

## Pandas setup

In [54]:
crcd = "Select * from dflt._ppko_crc_2019d_demographic_table"
colon_denom_df = pd.read_sql(sql=crcd, con=conn)
crcn = "Select * from dflt._ppko_crc_2019n_demographic_table"
colon_numer_df = pd.read_sql(sql=crcn, con=conn)

In [57]:
colon_denom_df.shape

(2375, 10)

In [59]:
colon_numer_df.shape  # Extra cols: EmergencyEventDate, EmergencyEventLocation, TypeOfEmergencyEvent

(572, 13)

## Analysis (colon)

In [66]:
colon_numer_df.describe()

,PatientAge
count,572.000000
mean,71.173077
std,12.161196
min,30.000000
25%,64.000000
50%,72.000000
75%,80.000000
max,97.000000


In [65]:
colon_denom_df.describe()

,PatientAge
count,2375.000000
mean,69.480421
std,10.922852
min,30.000000
25%,63.000000
50%,71.000000
75%,76.000000
max,98.000000


In [70]:
categorical_vars = [
    #'PatientSSN',
    #'DiagnosticEventDateTime',
    #'DiagnosisEventLocation',
    #'TypeOfCancer',
    'StageOfCancer',
    'TypeOfDiagnosisEvent',
    #'DeathDateTime',
    'PatientSex',
    #'PatientAge',
    'PatientRace'
]

for v in categorical_vars:
    display(colon_numer_df[v].value_counts())

IV         74
III        58
II         49
I          37
Unk/Uns    25
0          10
NA          3
Name: StageOfCancer, dtype: int64

REGISTRY ENTRY    572
Name: TypeOfDiagnosisEvent, dtype: int64

M    548
F     24
Name: PatientSex, dtype: int64

WHITE, NOT HISPANIC    392
BLACK                  113
HISPANIC                41
OTHER                   26
Name: PatientRace, dtype: int64

In [71]:
for v in categorical_vars:
    display(colon_denom_df[v].value_counts())

I          247
III        231
IV         186
II         184
Unk/Uns     83
0           65
NA          13
Name: StageOfCancer, dtype: int64

REGISTRY ENTRY    2375
Name: TypeOfDiagnosisEvent, dtype: int64

M    2263
F     112
Name: PatientSex, dtype: int64

WHITE, NOT HISPANIC    1619
BLACK                   494
HISPANIC                148
OTHER                   114
Name: PatientRace, dtype: int64